# Tutorial: Creating and Processing a Dataset for PyTorch

In [ ]:
from hrtfdata.torch.full import CIPIC, ARI, Listen, BiLi, ITA, HUTUBS, SADIE2, ThreeDThreeA, CHEDAR, Widespread, SONICOM
from hrtfdata.torch import collate_dict_dataset
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
base_dir = Path('../HRTF Datasets')

## Introduction

This tutorial assumes you have completed the tutorial "Using a dataset of planes in PyTorch" (available at `docs/tutorial-plane.ipynb`), so go read that first if you haven't done so already.

As mentioned in the previous tutorial, the purpose of `hrtfdata` is to provide PyTorch [`Datasets`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) for multiple collections in a unified programming interface. What is not immediately apparent from using the `XPlane` classes, is that you can do more than simply load predefined datasets. The idea behind `hrtfdata` is that you can select any information contained in an HRTF data collection and combine them into a PyTorch dataset with your desired characteristics. It can be helpful to think of the classes in `hrtfdata.torch.full` as `Dataset` _generators_ and the classes in `hrtfdata.pytorch.planar` as predefined configurations of those generators (with added helper functionality like plotting etc.).

The same data collections as for the plane datasets are available for use:
- CIPIC
- ARI
- Listen
- BiLi
- ITA
- HUTUBS
- SADIE II
- 3D3A
- CHEDAR
- Widespread
- SONICOM

Each of them has a corresponding class that can be loaded from `hrtfdata.torch.full`, as is done above.

## The concept of "specifications"

To create a `Dataset`, you need to define its "specification", meaning that you need to tell what information you want to use as `features` and optionally what as `target` labels and/or as `groups`. Each of these specifications takes the form of a Python dictionary with any combination of the following keys:

- collection: a string identifier of the name of the data collection
- subject: an integer giving the identifier of the subject in the collection
- side: a string indicating the side of the head 
- hrirs: an array containing the HRIRs for the given positions and side of the head

These four pieces of information are available for every collection. The following keys (and corresponding information) are sometimes available, depending on the collection, and will be discussed in a later tutorial:

- image: an array containing the pixel values of an image associated with a given side of the head 
- 3d-model: an array containing a 3D model
- anthropometry: an array containing anthropometric measurements

The values for each of these keys are also Python dictionaries themselves, and allow to pass parameters for each type of info. The first three types `collection`, `subject` and `side` have no parameters, so an empty dictionary `{}` should be passed as value for these keys. The `hrir` type does allow to pass parameters in the form of a dictionary with the following keys:

- domain: the HRIR/HRTF representation (`time`, `magnitude`, `magnitude_db`, `phase` or `complex`, default: `time`) 
- side: selecting a side of the head (`left`, `right`, `both`, `both-left`, `both-right`, `None`, default: `None` meaning any side available)
- row_angles: a list of angles in degrees to select from the fundamental coordinate plane (explained later), default: all available
- column_angles: a list of angles in degrees to select from the projection coordinate plane (explained later), default: all available

An example should make this clearer: the three `spec`s used in the creation of `XPlane` datasets roughly correspond to the following.

In [ ]:
domain = 'magnitude_db'
side = 'left'
ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'side': side, 'domain': domain}}, target_spec={'side': {}}, group_spec={'subject': {}})

## Dataset functionality

The resulting dataset has the a large part of the functionality of plane datasets, excluding plotting. So you can get its length, index individual data points as dictionaries and check their keys.

In [ ]:
isinstance(ds, Dataset), len(ds), ds[0].keys()

You can verify that the `target` and `group` value indeed contain the side of the head and the subject id of the data point, as requested.

In [ ]:
ds[0]['target'], ds[0]['group']

As a reminder, the dict format of the data points requires the use of the non-default `collate_dict_dataset` collation function when creating a `torch.util.data.DataLoader` (to convert the dataset into expected `(feature, target)` pairs), but also allows to split the dataset while keeping according to groups.

In [ ]:
DataLoader(ds, collate_fn=collate_dict_dataset)

Moreover, you can get the sample rate of the HRIR and its corresponding HRTF frequencies.

In [ ]:
ds.hrir_samplerate, ds.hrtf_frequencies

The ids of all available subjects and of this particular selection (all by default) can also be obtained using the properties `available_subject_ids` and `subject_ids`. The actual selection can be achieved by passing the argument `subject_ids` to the dataset constructor, just like for plane datasets (including the `first`, `last` and `random` options).

In [ ]:
ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'side': side, 'domain': domain}}, target_spec={'side': {}}, group_spec={'subject': {}}, subject_ids='random')
ds.available_subject_ids[:10], ds.subject_ids

## More specification examples

Only `feature_spec` is obligatory, `target_spec` and/or `group_spec` can be absent, leading to empty values, which is especially useful for situations where no target labels are needed (e.g. GANs).

In [ ]:
ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'side': side, 'domain': domain}}, subject_ids='first')
ds[0]['target'], ds[0]['group']

Multiple elements of information can be given in a `spec`, which will then be combined into one multi-dimensional element, for instance a target label consisting of three parts as below.

In [ ]:
ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {}}, target_spec={'collection': {}, 'subject': {}, 'side': {}}, subject_ids='first')
ds[0]['target']

HRIRs do not necessarily need to be used as features they can be used in any spec just like any other type of information. 

In [ ]:
ds = ARI(base_dir / 'ARI',  feature_spec={'side': {}}, target_spec={'hrirs': {}}, subject_ids='first')
ds[0]['features'], ds[0]['target'].shape, ds[0]['target'].dtype

The dtype of numerical values is `np.float32` by default, but this can be changed by passing a `dtype` argument to the constructor. When requesting an HRTF in the `complex` domain, it is necessary to specify a complex dtype too, otherwise an error will be thrown.

In [ ]:
try:
    ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'domain': 'complex'}}, subject_ids='first')
except ValueError as e:
    print(e)

In [ ]:
ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'domain': 'complex'}}, subject_ids='first', dtype=np.complex64)
ds[0]['features'].shape, ds[0]['features'].dtype

## Layout of HRIR data

Regardless of where the HRIRs are used, they are always stored as a 3-dimensional array. In order to describe the data layout independently of coordinate system, we need to define two terms. The _fundamental_ plane of a coordinate system is the reference plane onto which each point gets projected. Its angles span a range of 360 degrees. The _orthogonal_ plane of a coordinate system is the plane defined by a point, its projection and the origin. Its angles span a range of 180 degrees. In practice, the fundamental plane will nearly always be the horizontal plane because only the CIPIC collection uses interaural-polar coordinates, all other supported collections use vertical-polar coordinates.

The various spatial positions for which a HRIR measurement is available are organised along the rows and columns of the 3D array. The values of a single HRIR/HRTF (depending on the selected `domain`) for a particular position determined by row and column index are stored in the third dimension. The rows are ordered by increasing angle in the fundamental plane, constrained to the interval [-180, 180) (so vertical angles for CIPIC, azimuths for all the rest). The columns of the array are ordered by increasing angle in the orthogonal plane, constrained to the interval [-90, 90] (so lateral angles for CIPIC, elevations for all the rest).

This means that the spherical positions are stored as a [plate carrée](https://en.wikipedia.org/wiki/Equirectangular_projection) projection. The first and last column are always the positions closest to the poles, regardless of the coordinate system, although the measurements for the poles are not necessarily present (in the collection or just in a particular subselection).

In [ ]:
ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'side': side, 'domain': domain}}, target_spec={'side': {}}, group_spec={'subject': {}}, subject_ids='first')
ds[0]['features'].shape

The actual values of the angles can be read from the `row_angles` and `column_angles` properties.

In [ ]:
(len(ds.row_angles), len(ds.column_angles)), ds.row_angles, ds.column_angles

By default, all positions available in a collection are read, but a selection can be made by adding them to the `spec` as an iterable of angles. A value of `None` signifies all available, the default.

In [ ]:
ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'side': side, 'domain': domain, 'row_angles': None, 'column_angles': None}}, subject_ids='first')
ds[0]['features'].shape, ds.row_angles, ds.column_angles

You can combine a value of `None` for either `row_angles` or `column_angles` with an iterable of one or more angles, in which case the subset of all row/column angles with the given column/row angle(s) will be returned.

In [ ]:
ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'side': side, 'domain': domain, 'row_angles': (0,), 'column_angles': None}}, subject_ids='first')
ds[0]['features'].shape, ds.row_angles, ds.column_angles

If both `row_angles` and `column_angles` are given as iterables, they need to be of the same length and all spatial positions determined by the pairs of angles with equal index are returned.

In [ ]:
from itertools import product
angle_pairs = np.array(tuple(product(np.arange(-180, 180, 20), (-30, 0, 30))))
ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'side': side, 'domain': domain, 'row_angles': angle_pairs[:, 0], 'column_angles': angle_pairs[:, 1]}}, subject_ids='first')
ds[0]['features'].shape, ds.row_angles, ds.column_angles

Requesting an angle that is not available in the collection will be silently ignored. So when using the positional values in further processing, it is advised to use the exact angles that are returned by reading the relevant properties instead of using the requested values.

In [ ]:
requested_row_angles = (0, 0)
requested_column_angles = (0, 3.14)
ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'side': side, 'domain': domain, 'row_angles': requested_row_angles, 'column_angles': requested_column_angles}}, subject_ids='first')
ds[0]['features'].shape, ds.row_angles, ds.column_angles

In [ ]:
(np.unique(requested_row_angles) == ds.row_angles).all(), (np.unique(requested_column_angles) == ds.column_angles).all()

Only when none of the requested angle pairs are available, will an exception be thrown.

In [ ]:
try:
    ds = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'side': side, 'domain': domain, 'row_angles': (0,), 'column_angles': (3.14,)}}, subject_ids='first')
except ValueError as e:
    print(e)

If we think of the row and column angles as forming a matrix of available positions, then this matrix is not necessarily dense. Not every collection has chosen a spatial sampling where every row/column angle combination is measured. Consequently, the 3D array of HRIRs is stored as a [NumPy masked array](https://numpy.org/doc/stable/reference/maskedarray.html), where row/column combinations that have not been measured are masked.

We can visualise the distribution of the spatial positions by plotting the mask of a collection (technically a single, third dimension slice of the mask but all slices are the same since either all samples of a HRIR are present or none). Do note that the masks are displayed by position index, not actual angles, and that the angular sampling is not necessarily regular. Therefore these plots cannot be interpreted as representations of the angular distribution of the measurement positions.

In [ ]:
collections = (
    (CIPIC, base_dir / 'CIPIC'),
    (ARI, base_dir / 'ARI'),
    (Listen, base_dir / 'Ircam Listen'),
    (BiLi, base_dir / 'Ircam BiLi'),
    (ITA, base_dir / 'ITA Aachen'), 
    (HUTUBS, base_dir / 'HUTUBS'),
    (SADIE2, base_dir / 'SADIE II'),
    (ThreeDThreeA, base_dir / '3D3A'),
    (CHEDAR, base_dir / 'CHEDAR'),
    (Widespread, base_dir / 'Widespread'),
    (SONICOM, base_dir / 'SONICOM'),
)

from math import ceil
fig = plt.figure(figsize=(16, 10))
for idx, (collection, data_dir) in enumerate(collections):
    ds = collection(data_dir,  feature_spec={'hrirs': {}}, subject_ids='first')
    ax = fig.add_subplot(2, ceil(len(collections)/2), idx+1)
    ax.set_title(collection.__name__)
    ax.matshow(np.ma.getmaskarray(ds[0]['features'][:, :, 0, 0]))
    ax.set_aspect(0.5)

## Processing HRIRs further

It is possible to process the HRIRs further without needing to modify any internal code. To that end, each dataset constructor accepts a callable as argument `hrir_transform`. The callable itself takes one argument, the 3D array of HRIRs and needs to return the modified array. For instance, suppose that dB conversion of the HRTF magnitudes was not built-in, then it could be added by passing the function below.

In [ ]:
def db_calc(hrtfs):
    return 20*np.log10(np.clip(hrtfs, 3.305e-7, None))

external_db_calc = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'domain': 'magnitude'}}, subject_ids='first', hrir_transform=db_calc)
builtin_db_calc = ARI(base_dir / 'ARI',  feature_spec={'hrirs': {'domain': 'magnitude_db'}}, subject_ids='first')
np.allclose(external_db_calc[0]['features'], builtin_db_calc[0]['features'])

The returned HRIRs do not necessarily need to have the same shape as the input. For instance, this is done by the `XPlane` classes, which return 2D planes. Internally, the `plane` argument of the constructor is converted into a set of row and column angles, which get read and subsequently stitched together into a single plane to form the output.